# Optimized Embedded GPU Intelligence Extraction - SageMaker Studio

This notebook runs the Streamlit app with **fully optimized in-process GPU inference**.

**Branch:** `embedded-microbatch` (New optimized version!)

---

## 🚀 Performance Optimizations:
- ⚡ **FlashAttention-2** with SDPA fallback (2-3x speedup)
- 🔥 **torch.compile** graph optimization (PyTorch 2.0+)
- 📦 **Micro-batching** with length-aware bucketing
- 🎯 **Pinned memory** + dual CUDA streams
- 🧱 **4-bit quantization** (14-18GB VRAM vs 28GB)
- 🎮 **90-98% GPU utilization** (vs 98% @ 0.1 profiles/sec before)
- 🌐 **Public URL** via ngrok

---

## 📊 Expected Performance (L4 GPU):
- **Speed**: 2-5 profiles/sec (vs 0.1 baseline)
- **Speedup**: 20-50x faster than previous version
- **Time for 3,200 profiles**: 10-25 minutes (vs 8+ hours)

---

## Requirements:
- **GPU**: ml.g4dn.xlarge or better (L4, V100, A10G recommended)
- **VRAM**: 16GB+ (22GB+ optimal)
- **Disk**: ~5GB for model cache
- **CUDA**: 11.8+

## 1. Install Dependencies

In [ ]:
%%time
print("="*80)
print("📦 INSTALLING DEPENDENCIES")
print("="*80)

# Core dependencies
print("\n1️⃣ Installing core packages...")
!pip install -q streamlit pyngrok pandas pydantic pydantic-settings python-dotenv py-spy

# PyTorch and transformers
print("2️⃣ Installing PyTorch 2.1+ with CUDA...")
!pip install -q torch>=2.1.0 transformers>=4.43.0 accelerate bitsandbytes einops

# FlashAttention-2 (2-3x speedup!)
print("3️⃣ Installing FlashAttention-2 (this may take 2-3 minutes)...")
print("   If this fails, the system will automatically fall back to SDPA (still fast)")
try:
    !pip install -q flash-attn --no-build-isolation
    print("   ✅ FlashAttention-2 installed successfully!")
except Exception as e:
    print(f"   ⚠️  FlashAttention-2 install failed (will use SDPA fallback): {e}")

print("\n" + "="*80)
print("✅ ALL DEPENDENCIES INSTALLED!")
print("="*80)
print("Installed:")
print("  - Streamlit, pyngrok, pandas, pydantic")
print("  - PyTorch 2.1+, transformers, accelerate")
print("  - bitsandbytes (4-bit quantization)")
print("  - einops, py-spy")
print("  - FlashAttention-2 (if successful)")
print("="*80)

## 2. Configure Ngrok Token

**Get your free token:** https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
# REPLACE THIS WITH YOUR ACTUAL NGROK TOKEN
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"

if NGROK_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("❌ ERROR: Set your ngrok token above!")
    print("   Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
else:
    print(f"✅ Ngrok token configured")

## 3. Pre-Download Qwen2.5-7B Model (First Run Only)

This downloads and caches the model (~4.3 GB). Subsequent runs will be instant.

**Time:** 5-10 minutes on first run, instant after that.

In [ ]:
%%time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("="*80)
print("📥 PRE-DOWNLOADING QWEN2.5-7B-INSTRUCT MODEL")
print("="*80)

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
USE_4BIT = True

print(f"\n📦 Model: {MODEL_NAME}")
print(f"🔧 4-bit quantization: {USE_4BIT}")
print(f"💾 Cache: ~/.cache/huggingface/hub/")
print(f"🖥️  Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

if torch.cuda.is_available():
    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  No GPU detected - model will run on CPU (slower)")

print("\n" + "="*80)
print("STEP 1: Downloading Tokenizer")
print("="*80)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
print("✅ Tokenizer downloaded (~1 MB)")

print("\n" + "="*80)
print("STEP 2: Downloading Model Weights (~4.3 GB)")
print("="*80)
print("⏳ This will take 5-10 minutes on first run...")
print("   Subsequent runs will be instant (cached)\n")

if USE_4BIT:
    from transformers import BitsAndBytesConfig
    print("🧱 Loading with 4-bit quantization (bitsandbytes)")
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quant_config,
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    print("✅ Model downloaded and quantized to 4-bit")
    print(f"   Memory usage: ~2.7 GB")
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    print("✅ Model downloaded (FP16)")
    print(f"   Memory usage: ~14 GB")

print("\n" + "="*80)
print("STEP 3: Testing Generation")
print("="*80)

inputs = tokenizer("Hello, I am", return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.7)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ Test generation successful!")
print(f"   Input: 'Hello, I am'")
print(f"   Output: '{result}'")

# Clean up to free memory
del model
del tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n" + "="*80)
print("✅ MODEL READY FOR EMBEDDED GPU INFERENCE!")
print("="*80)
print(f"✅ Model: {MODEL_NAME}")
print(f"✅ Quantization: {'4-bit (NF4)' if USE_4BIT else 'FP16'}")
print(f"✅ Cache: ~/.cache/huggingface/hub/")
print(f"✅ Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print("\nStreamlit app will use this cached model for instant startup! 🚀")
print("="*80)

## 4. Setup SQLite Database

In [ ]:
import sqlite3

print("="*80)
print("⚙️  CONFIGURING OPTIMIZED GPU SETTINGS")
print("="*80)

# Create .env file with OPTIMIZED settings
with open('.env', 'w') as f:
    f.write('''DATABASE_URL=sqlite:////tmp/contributor_intelligence.db

# Embedded GPU Model
EMBED_MODEL=Qwen/Qwen2.5-7B-Instruct
EMBED_4BIT=1                 # 1=4bit quantization (7GB VRAM, slower), 0=FP16 (14GB VRAM, 2-3x faster!)
EMBED_DTYPE=float16          # Used if EMBED_4BIT=0 (float16 recommended for speed)
MAX_TOKENS=320
TEMPERATURE=0.05
TOP_P=0.9

# 🚀 OPTIMIZED GPU CONFIGURATION (Critical for performance!)
MAX_CONCURRENT_LLM=1         # Worker threads (MUST be 1 for proper batching!)
INFER_CONCURRENCY=10         # Max concurrent GPU batches (semaphore) - INCREASED for L4!
MICRO_BATCH_SIZE=31          # Prompts per batch (optimal for L4 GPU)
BATCH_LATENCY_MS=50          # Wait time to collect batch (ms) - lower for faster response
PREFILL_BATCH_TOKENS=4096    # Max input tokens per prefill
DECODE_CONCURRENCY=8         # Max concurrent decode ops
USE_FLASH_ATTENTION=True     # Enable FlashAttention-2 (auto-fallback)
ENABLE_COMPILE=True          # Enable torch.compile optimization
EXTRACTION_WORKERS=50        # App-level threads for streaming extraction (need many to keep queue full!)

# Logging
LOG_LEVEL=INFO
''')

# ✅ CRITICAL: Load environment variables into current process immediately!
from dotenv import load_dotenv
load_dotenv(override=True)

print("✅ Environment configured and LOADED with optimized GPU settings")
print("\n📊 Actual Configuration (loaded from .env):")
import os
print(f"   MAX_CONCURRENT_LLM={os.getenv('MAX_CONCURRENT_LLM', 'N/A')}     ← Worker threads")
print(f"   INFER_CONCURRENCY={os.getenv('INFER_CONCURRENCY', 'N/A')}    ← Concurrent GPU batches")
print(f"   MICRO_BATCH_SIZE={os.getenv('MICRO_BATCH_SIZE', 'N/A')}     ← Prompts per batch")
print(f"   BATCH_LATENCY_MS={os.getenv('BATCH_LATENCY_MS', 'N/A')}      ← Collection window (ms)")
print(f"   EXTRACTION_WORKERS={os.getenv('EXTRACTION_WORKERS', 'N/A')}  ← App-level threads")
print(f"   USE_FLASH_ATTENTION={os.getenv('USE_FLASH_ATTENTION', 'N/A')}")
print(f"   ENABLE_COMPILE={os.getenv('ENABLE_COMPILE', 'N/A')}")

# Create SQLite database
print("\n" + "="*80)
print("🗄️  SETTING UP SQLITE DATABASE")
print("="*80)

db_path = '/tmp/contributor_intelligence.db'
conn = sqlite3.connect(db_path)
conn.execute('''
    CREATE TABLE IF NOT EXISTS contributors (
        email TEXT PRIMARY KEY,
        contributor_id TEXT UNIQUE NOT NULL,
        processed_data TEXT NOT NULL,
        intelligence_summary TEXT,
        processing_status TEXT DEFAULT 'pending',
        error_message TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        intelligence_extracted_at TIMESTAMP
    )
''')
conn.execute('CREATE INDEX IF NOT EXISTS idx_status ON contributors(processing_status)')
conn.commit()
conn.close()

print(f"✅ SQLite database: {db_path}")
print("\n" + "="*80)
print("✅ SETUP COMPLETE!")
print("="*80)

## 5. Configure Ngrok

In [ ]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = NGROK_TOKEN
print("✅ Ngrok configured")

## 6. Start Streamlit App

In [ ]:
import subprocess
import time

# Stop any existing Streamlit
subprocess.run(['pkill', '-f', 'streamlit'], capture_output=True)
time.sleep(2)

# Start Streamlit
print("🎨 Starting Streamlit app with embedded GPU...")
with open('/tmp/streamlit.log', 'w') as log:
    streamlit_process = subprocess.Popen(
        ['streamlit', 'run', 'app.py', 
         '--server.port', '8501',
         '--server.headless', 'true'],
        stdout=log,
        stderr=log
    )

time.sleep(8)
print(f"✅ Streamlit started (PID: {streamlit_process.pid})")
print("⏳ Model loading in background (first request will take ~30s)...")

## 7. Create Public URL 🌐

In [ ]:
import os
from dotenv import load_dotenv

try:
    ngrok.kill()
    time.sleep(2)
    public_url = ngrok.connect(8501, bind_tls=True)
    
    # Load config to show actual values
    load_dotenv()
    
    print("\n" + "="*80)
    print("🌐 PUBLIC URL READY!")
    print("="*80)
    print(f"\n🔗 {public_url}")
    print("\n" + "="*80)
    print("🚀 OPTIMIZED GPU FEATURES:")
    print("="*80)
    print("✅ FlashAttention-2 with SDPA fallback (2-3x speedup)")
    print("✅ torch.compile graph optimization")
    print("✅ Pinned memory + dual CUDA streams")
    print("✅ 4-bit quantization (14-18GB VRAM)")
    print(f"✅ Micro-batching (size={os.getenv('MICRO_BATCH_SIZE', '6')})")
    print(f"✅ GPU concurrency (slots={os.getenv('INFER_CONCURRENCY', '3')})")
    print(f"✅ Worker threads={os.getenv('MAX_CONCURRENT_LLM', '1')} (optimized!)")
    print("\n" + "="*80)
    print("📊 EXPECTED PERFORMANCE:")
    print("="*80)
    print("Speed: 2-5 profiles/sec (vs 0.1 baseline)")
    print("Time for 3,200: 10-25 minutes (vs 8+ hours)")
    print("GPU Utilization: 90-98%")
    print("VRAM: 16-18 GB / 22 GB")
    print("\n⚠️  Keep this cell running to maintain the ngrok tunnel!")
    print("="*80)
    
except Exception as e:
    print(f"❌ Error: {e}")

## 8. Stop Services

In [ ]:
import torch

ngrok.kill()
subprocess.run(['pkill', '-f', 'streamlit'], capture_output=True)
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✅ All services stopped")